In [2]:
!pip install -r requirements.txt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------------------------------ --- 2.6/2.9 MB 13.8 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.6 MB ? eta -:--:--
   - -------------------------------------- 2.9/101.6 MB 15.2 MB/s eta 0:00:07
   -- ------------------------------------- 6.6/101.6 MB 16.8 MB/s eta 0:00:06
   --- ------------------------------------ 10.0/101.6 MB 16.8 MB/s eta 0:00:06
   ----- ---------------------------------- 14.2/101.6 MB 17.8 MB/s eta 0:00:05
   ------ --------------------------------- 16.5/101.6 MB 16.3 MB/s eta 0:00:06
   -------- ------------------------------- 20.4/101.6 MB 16.6 MB/s eta 0:00:05
   --------- ------------------------------ 24.6/10

  DEPRECATION: Building 'antlr4-python3-runtime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'antlr4-python3-runtime'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
federated-language 0.4.0 requires numpy~=2.0, but you have numpy 1.26.4 which is incompatible.
federated-language 0.4.0 requires protobuf~=5.26, but you have protobuf 4.25.7 which is incompatible.


In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from utils.utils import *
from utils.LLM import LLM_pretrained, LLM_cen_partial
from utils.LLM import get_fireworks_api_key,load_env

function 'cadam32bit_grad_fp32' not found


In [4]:
# Load config
cfg = get_config("centralized")

# Inspect the config
print_config(cfg)

dataset:
  name: medalpaca/medical_meadow_medical_flashcards
model:
  name: EleutherAI/pythia-70m
  quantization: 4
  gradient_checkpointing: true
  use_fast_tokenizer: true
  lora:
    peft_lora_r: 16
    peft_lora_alpha: 64
    target_modules: null
train:
  save_every_round: 5
  seq_length: 512
  padding_side: left
  training_arguments:
    learning_rate: 0.0005
    per_device_train_batch_size: 2
    gradient_accumulation_steps: 1
    logging_steps: 1
    max_steps: 5
    report_to: null
    save_steps: 200
    save_total_limit: 10
    gradient_checkpointing: true
    lr_scheduler_type: cosine



In [ ]:
trainset_full = load_dataset(cfg.dataset.name, split='train')
train_test = trainset_full.train_test_split(test_size=0.9, seed=1234)
train_dataset = train_test["train"]

train_dataset = format_dataset(train_dataset)

print(train_dataset)

In [ ]:
# Print an example from the dataset
example_index = 9

data_point = train_dataset[example_index]

data_point

In [ ]:
llm_pretrained = LLM_pretrained()

In [ ]:
prompt = "How to predict the weather"
llm_pretrained.eval(prompt, verbose = False)
llm_pretrained.print_response(verbose = False)

In [ ]:
llm_pretrained.eval(data_point['instruction'], verbose=True)

llm_pretrained.print_response()

ex_response = format_string(data_point['response'])
print(f"Expected output:\n\t{ex_response}")

In [ ]:
model = get_model(cfg.model)


trainable, all_parameters = model.get_nb_trainable_parameters()
print(f"Trainable parameters: {trainable}")
print(f"All parameters: {all_parameters}")
print(f"Trainable (%): {100*trainable / all_parameters:.3f}")

In [ ]:
(
tokenizer, 
data_collator, 
format_prompts_fn
) = get_tokenizer_and_data_collator_and_propt_formatting(
        cfg.model.name, cfg.model.use_fast_tokenizer, cfg.train.padding_side
)

In [ ]:
save_centralized = "./my_centralized_model"
def finetune_centralised():

    # The notebooks you are running in this course does not
    # come with a GPU. However, we don't harcode "cpu" as the
    # device to use in case you wish to download this notebook
    # and run it on your own GPU.
    use_cuda = torch.cuda.is_available()
    training_arguments = TrainingArguments(
                         **cfg.train.training_arguments,
                         use_cpu=not(use_cuda),
                         output_dir=save_centralized,
    )

    # Construct trainer
    trainer = SFTTrainer(
        tokenizer=tokenizer,
        data_collator=data_collator,
        formatting_func=format_prompts_fn,
        max_seq_length=cfg.train.seq_length,
        model=model,
        args=training_arguments,
        train_dataset=train_dataset,
    )

    # Do local training
    trainer.train()

    # Save the checkpoint
    model.save_pretrained(save_centralized)

In [ ]:
finetune_centralised()

In [ ]:
llm_cen = LLM_cen_partial()
example_index = 9

data_point = train_dataset[example_index]

llm_cen.eval(data_point['instruction'], verbose=True)

llm_cen.print_response()

ex_response = format_string(data_point['response'])
print(f"Expected output:\n\t{ex_response}")

In [ ]:
visualize_results(results=['7b/pretrained', '7b/cen_10'])